In [1]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
import cv2

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer


app = Flask(__name__)


MODEL_PATH = 'my_model.h5'
# Load your trained model
model = load_model(MODEL_PATH)

class_names = ['fire_images', 'non_fire_images']
def model_predict(img_path, model):
    image = cv2.imread(img_path)
    # Preprocessing the image
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = np.expand_dims(image, axis=0)
    preds = model.predict(image)
    preds = np.where(preds >= 0.5, 1, 0)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        basepath = os.getcwd()
        file_path = os.path.join(basepath, secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)

        # Process your result for human
        result = str(f'{class_names[preds[0][0]]}')
        return result
    return None

if __name__ == '__main__':
      app.run(host='127.0.0.1', port=5000)




 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2024 15:55:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2024 15:55:09] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2024 15:55:09] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2024 15:55:18] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 1s 623ms/step


127.0.0.1 - - [20/Jan/2024 15:55:28] "POST /predict HTTP/1.1" 200 -
